# Test Case Generator
Generate unit tests for any programming language using AI models.
Works on both local Jupyter and Google Colab.

In [ ]:
# Environment detection and setup
import sys

# Detect if running in Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print('Running in Google Colab')
    # Install dependencies
    !pip install -q openai anthropic google-generativeai transformers gradio python-dotenv accelerate bitsandbytes
else:
    print('Running locally')

print('Environment detected')

In [ ]:
# All imports
import os
import re
import gc
import torch
import requests
import subprocess

# Frontier models
from openai import OpenAI
import anthropic
import google.generativeai as genai

# Open source
if not IN_COLAB:
    import ollama
else:
    ollama = None

try:
    import groq
except ImportError:
    groq = None

# Transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig

# UI
import gradio as gr

print('Imports loaded')

In [ ]:
# Load API keys based on environment

if IN_COLAB:
    # Use Colab secrets
    from google.colab import userdata
    
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    HUGGINGFACE_API_KEY = userdata.get('HUGGINGFACE_API_KEY')
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')
    print('Using Colab secrets')
else:
    # Use .env file
    from dotenv import load_dotenv
    load_dotenv()
    
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
    GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
    HUGGINGFACE_API_KEY = os.getenv('HUGGINGFACE_API_KEY')
    GROQ_API_KEY = os.getenv('GROQ_API_KEY')
    print('Using .env file')

# Initialize clients
openai_client = OpenAI(api_key=OPENAI_API_KEY) if OPENAI_API_KEY else None
anthropic_client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY) if ANTHROPIC_API_KEY else None
if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)

# Check GPU availability
if torch.cuda.is_available():
    print(f'GPU available: {torch.cuda.get_device_name(0)}')
else:
    print('No GPU available, using CPU')

print('Environment configured')

In [ ]:
# Model catalog
MODEL_TYPES = ['GPT', 'Claude', 'Gemini', 'HuggingFace']

MODEL_NAMES = {
    'GPT': ['gpt-4o-mini', 'gpt-4o', 'gpt-3.5-turbo'],
    'Claude': ['claude-3-5-haiku-latest', 'claude-3-5-sonnet-latest'],
    'Gemini': ['gemini-2.0-flash-exp', 'gemini-1.5-pro', 'gemini-1.5-flash'],
    'HuggingFace': [
        'meta-llama/Llama-3.1-3B-Instruct',
        'microsoft/bitnet-b1.58-2B-4T',
        'tiiuae/Falcon-E-3B-Instruct',
        'Qwen/Qwen2.5-7B-Instruct'
    ]
}

CATEGORY_TO_PROVIDER = {
    'GPT': 'openai',
    'Claude': 'anthropic',
    'Gemini': 'google',
    'HuggingFace': 'huggingface'
}

print('Model catalog ready')

In [ ]:
# System prompt
SYSTEM_PROMPT = (
    'You are an expert test engineer. Generate runnable unit tests for the provided code. '
    'Requirements:\n'
    '- Include original code FIRST, then unit tests\n'
    '- Add brief comments explaining each test\n'
    '- Output ONLY code (no markdown fences, no extra text)\n'
    '- Make it runnable as-is (include main/runner if needed)\n'
    '- Keep tests deterministic (no network/filesystem/external deps unless mocked)\n'
    '- Cover happy path, negative cases, and boundaries'
)

print('System prompt ready')

In [ ]:
# Helper functions

def resolve_provider_and_model(category, selected_model=None):
    provider = CATEGORY_TO_PROVIDER.get(category)
    if not provider:
        raise ValueError(f'Unknown category: {category}')
    model_list = MODEL_NAMES.get(category, [])
    model = selected_model if selected_model else (model_list[0] if model_list else None)
    return provider, model

def extract_code_and_lang(text):
    text = text.strip()
    match = re.match(r'^```(\w+)?\s*\n([\s\S]*?)\n```$', text, re.MULTILINE)
    if match:
        lang = (match.group(1) or '').lower()
        code = match.group(2)
        return code, lang if lang else None
    return text, None

def quant_config():
    try:
        return BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_quant_type='nf4'
        )
    except:
        return None

print('Helpers ready')

In [ ]:
# Provider response functions

def get_chatgpt_response(prompt, model_name, output_tokens=1024):
    if not openai_client:
        raise RuntimeError('OpenAI API key not configured')
    messages = [{'role': 'system', 'content': SYSTEM_PROMPT}, {'role': 'user', 'content': prompt}]
    resp = openai_client.chat.completions.create(
        model=model_name,
        messages=messages,
        max_tokens=output_tokens
    )
    return resp.choices[0].message.content

def get_claude_response(prompt, model_name, output_tokens=1024):
    if not anthropic_client:
        raise RuntimeError('Anthropic API key not configured')
    resp = anthropic_client.messages.create(
        model=model_name,
        max_tokens=output_tokens,
        system=SYSTEM_PROMPT,
        messages=[{'role': 'user', 'content': prompt}]
    )
    return resp.content[0].text

def get_gemini_response(prompt, model_name, output_tokens=1024):
    if not GOOGLE_API_KEY:
        raise RuntimeError('Google API key not configured')
    model = genai.GenerativeModel(model_name=model_name, system_instruction=SYSTEM_PROMPT)
    resp = model.generate_content(
        prompt,
        generation_config={'max_output_tokens': output_tokens, 'temperature': 0.7}
    )
    return resp.text

def get_hf_response(prompt, model_name, output_tokens=1024):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if not tokenizer.pad_token:
        tokenizer.pad_token = tokenizer.eos_token
    
    full_prompt = f'{SYSTEM_PROMPT}\n\n{prompt}'
    try:
        inputs = tokenizer.apply_chat_template([{'role': 'user', 'content': full_prompt}], return_tensors='pt')
    except:
        inputs = tokenizer(full_prompt, return_tensors='pt')['input_ids']
    
    if torch.cuda.is_available():
        inputs = inputs.to('cuda')
    
    if 'Falcon' in model_name:
        model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto', torch_dtype=torch.float16)
    elif 'bitnet' in model_name:
        model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto', trust_remote_code=True)
    else:
        model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto', quantization_config=quant_config())
    
    outputs = model.generate(inputs, max_new_tokens=output_tokens)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    del model, tokenizer, inputs, outputs
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    return response

print('Providers ready')

In [ ]:
# Gradio UI

def predict(category, model, prompt, num_tests, output_tokens):
    provider, resolved_model = resolve_provider_and_model(category, model)
    final_prompt = f'Generate {int(num_tests)} unit tests.\n\nUser requirements:\n{prompt}'
    
    if provider == 'openai':
        response = get_chatgpt_response(final_prompt, resolved_model, output_tokens)
    elif provider == 'anthropic':
        response = get_claude_response(final_prompt, resolved_model, output_tokens)
    elif provider == 'google':
        response = get_gemini_response(final_prompt, resolved_model, output_tokens)
    elif provider == 'huggingface':
        response = get_hf_response(final_prompt, resolved_model, output_tokens)
    else:
        return gr.update(value='Unsupported provider', language=None)
    
    code, lang = extract_code_and_lang(response)
    return gr.update(value=code, language=lang)

with gr.Blocks(title='Test Case Generator') as demo:
    gr.Markdown('# Test Case Generator')
    gr.Markdown('Generate unit tests for any code using AI models')
    
    with gr.Row():
        category_dd = gr.Dropdown(choices=MODEL_TYPES, value='GPT', label='Model Category')
        model_dd = gr.Dropdown(choices=MODEL_NAMES['GPT'], label='Model', value=MODEL_NAMES['GPT'][0])
    
    with gr.Row():
        num_tests_nb = gr.Number(value=5, precision=0, label='Number of tests', minimum=1, maximum=20)
        tokens_sld = gr.Slider(minimum=128, maximum=4096, value=1024, step=128, label='Max output tokens')
    
    prompt_tb = gr.Textbox(lines=10, label='Your code / requirements', placeholder='Paste your code here...')
    generate_btn = gr.Button('Generate Tests', variant='primary')
    output_code = gr.Code(label='Generated Tests', language=None, lines=20)
    
    def on_category_change(cat):
        models = MODEL_NAMES.get(cat, [])
        return gr.update(choices=models, value=models[0] if models else None)
    
    category_dd.change(on_category_change, inputs=category_dd, outputs=model_dd)
    
    def on_generate(cat, mdl, prmpt, num, toks):
        try:
            return predict(cat, mdl, prmpt, num, toks)
        except Exception as e:
            return gr.update(value=f'Error: {str(e)}', language=None)
    
    generate_btn.click(
        on_generate,
        inputs=[category_dd, model_dd, prompt_tb, num_tests_nb, tokens_sld],
        outputs=output_code
    )

print('UI ready')

In [ ]:
# Launch (auto-detects environment)
if IN_COLAB:
    demo.launch(share=True, debug=True)  # Public shareable link for Colab
else:
    demo.launch()  # Local server